# A full SimPy model

In this notebook, we will now build a full `simpy` process model. Our example is a queuing model of a 111 call centre.  We will include random arrivals and resources. We will keep this simple, and gradually add in detail and flexibility to our design.

## 1. Imports

In [1]:
import simpy
import numpy as np
import itertools

## 2. Problem background

Call operators in an 111 (urgent care) service receive calls at a rate of 100 per hour. Call length can be represented by a triangular distribution.  Calls last between 5 minutes and 15 minutes. Most calls last 7 minutes. There are 13 call operators.

![Call Centre Diagram](./img/callcentre.png)

## 3. SimPy resources

To model the call centre, we need to introduce a **resource**.

These resources represent the call operators. If a resource is not available then a process will **pause** (i.e. callers will wait for an operator to become available). 

We create a resource as follows:

```python
operators = simpy.Resource(env, capacity=20)
```

When we want to request a resource in our process, we create a `with` block as follows:

```python
with operators.request() as req:
    yield req
```

This tells SimPy that **your process needs an operator resource to progress**.  The code will pause until a resource is yielded. This gives us our **queuing effect**.  If a resource is not available immediately then the process will wait until one becomes available.

## 4. The service function

We will first create a python function called `service()` to simulate the service process for a call operator. We need to include the following logic:

1. **Request and wait** (if necessary) for a call operator.
2. Undergo **phone triage** (a delay). This is a sample from the Triangular distribution.
3. **Exit the system**.

Each caller that arrives in the simulation will this function as a SimPy **process**. As inputs to the function, we will pass:

* A unique patient identifier (`identifier`)
* A pool of operator resources (`operators`)
* The environment (`env`)

In [2]:
def service(identifier, operators, env, service_rng):
    '''
    Simulates the service process for a call operator

    1. request and wait for a call operator
    2. phone triage (triangular)
    3. exit system
    
    Params:
    ------
    
    identifier: int 
        A unique identifer for this caller
        
    operators: simpy.Resource
        The pool of call operators that answer calls
        These are shared across resources.
        
    env: simpy.Environment
        The current environent the simulation is running in
        We use this to pause and restart the process after a delay.

    service_rng: numpy.random.Generator
        The random number generator used to sample service times
    
    '''
    # record the time that call entered the queue
    start_wait = env.now

    # request an operator
    with operators.request() as req:
        yield req

        # record the waiting time for call to be answered
        waiting_time = env.now - start_wait
        print(f'operator answered call {identifier} at ' \
              + f'{env.now:.3f}')

        # sample call duration.
        call_duration = service_rng.triangular(left=5.0, mode=7.0,
                                               right=10.0)
        
        # schedule process to begin again after call_duration
        yield env.timeout(call_duration)

        # print out information for patient.
        print(f'call {identifier} ended {env.now:.3f}; ' \
              + f'waiting time was {waiting_time:.3f}')

## 5. The generator function

The generator function is very similar to the [pharamacy example](./03b_exercise1_solutions.ipynb).  

> ✂️ **Notice the pattern**. For most models you can just cut, paste and modify code you have used before.

In [3]:
def arrivals_generator(env, operators):
    '''
    Simulates the call arrival process and spawns 
    Inter-arrival time (IAT) is exponentially distributed

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    operators: simpy.Resource
        the pool of call operators.
    '''
    # create the arrival process rng 
    arrival_rng = np.random.default_rng()
    
    # create the service rng that we pass to each service process created
    service_rng = np.random.default_rng()
    
    # use itertools as it provides an infinite loop 
    # with a counter variable that we can use for unique Ids
    for caller_count in itertools.count(start=1):

        # 100 calls per hour (sim time units = minutes). 
        inter_arrival_time = arrival_rng.exponential(60/100)
        yield env.timeout(inter_arrival_time)

        print(f'call arrives at: {env.now:.3f}')

        # create a new simpy process for serving this caller.
        # we pass in the caller id, the operator resources, env, and the rng
        env.process(service(caller_count, operators, env, service_rng))

## 6. Run the model

In [4]:
# model parameters
RUN_LENGTH = 100
N_OPERATORS = 13

# create simpy environment and operator resources
env = simpy.Environment()
operators = simpy.Resource(env, capacity=N_OPERATORS)

env.process(arrivals_generator(env, operators))
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

call arrives at: 0.031
operator answered call 1 at 0.031
call arrives at: 1.651
operator answered call 2 at 1.651
call arrives at: 2.135
operator answered call 3 at 2.135
call arrives at: 2.161
operator answered call 4 at 2.161
call arrives at: 2.693
operator answered call 5 at 2.693
call arrives at: 4.102
operator answered call 6 at 4.102
call arrives at: 4.718
operator answered call 7 at 4.718
call arrives at: 5.552
operator answered call 8 at 5.552
call arrives at: 5.685
operator answered call 9 at 5.685
call arrives at: 5.784
operator answered call 10 at 5.784
call arrives at: 5.846
operator answered call 11 at 5.846
call arrives at: 5.993
operator answered call 12 at 5.993
call arrives at: 6.241
operator answered call 13 at 6.241
call 1 ended 6.273; waiting time was 0.000
call arrives at: 6.628
operator answered call 14 at 6.628
call arrives at: 6.954
call arrives at: 7.788
call arrives at: 7.992
call arrives at: 8.269
call 2 ended 8.492; waiting time was 0.000
operator answered c